In [ ]:
import pandas as pd
import numpy as np
import geopandas as gp

In [ ]:
households_csv = '~/Data/UKDA-5340-tab/tab/householdeul2017.tab'
individuals_csv = '~/Data/UKDA-5340-tab/tab/individualeul2017.tab'
trips_csv ='~/Data/UKDA-5340-tab/tab/tripeul2017.tab'
"https://geoportal.statistics.gov.uk/datasets/counties-and-unitary-authorities-december-2017-full-extent-boundaries-in-uk-wgs84?geometry=-1.922%2C50.793%2C2.379%2C51.397&page=2"
areas_shp = '~/Data/UKDA-5340-tab/Counties_and_Unitary_Authorities__December_2017__Boundaries_in_the_UK__WGS84'

## Load households data

In [3]:
hh_in = pd.read_csv(
    households_csv,
    sep='\t',
    usecols=['HouseholdID', 'SurveyYear', 'PSUID', 'W2', 'OutCom_B02ID',
       'HHIncome2002_B02ID', 'AddressType_B01ID', 'Ten1_B02ID',
       'Landlord_B01ID', 'ResLength_B01ID', 'HHoldCountry_B01ID',
       'HHoldGOR_B02ID', 'HHoldNumAdults', 'HHoldNumChildren',
       'HHoldNumPeople', 'HHoldStruct_B02ID', 'NumLicHolders',
       'HHoldEmploy_B01ID', 'NumVehicles', 'NumBike', 'NumCar', 'NumMCycle',
       'NumVanLorry', 'NumCarVan', 'WalkBus_B01ID', 'Getbus_B01ID',
       'WalkRail_B01ID', 'WalkRailAlt_B01ID',
       'HRPWorkStat_B02ID', 'HRPSEGWorkStat_B01ID', 'HHoldOAClass2011_B03ID',
       'Settlement2011EW_B03ID', 'Settlement2011EW_B04ID'],
)

hh_in.HHIncome2002_B02ID = pd.to_numeric(hh_in.HHIncome2002_B02ID, errors='coerce')
hh_in.NumLicHolders = pd.to_numeric(hh_in.NumLicHolders, errors='coerce')
hh_in.NumVehicles = pd.to_numeric(hh_in.NumVehicles, errors='coerce')
hh_in.NumCar = pd.to_numeric(hh_in.NumCar, errors='coerce')
hh_in.NumMCycle = pd.to_numeric(hh_in.NumMCycle, errors='coerce')
hh_in.NumVanLorry = pd.to_numeric(hh_in.NumVanLorry, errors='coerce')
hh_in.NumCarVan = pd.to_numeric(hh_in.NumCarVan, errors='coerce')
hh_in.Settlement2011EW_B04ID = pd.to_numeric(hh_in.Settlement2011EW_B04ID, errors='coerce')

hh_in.head()

/Users/fred.shone/.pyenv/versions/jupyter3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (23,35,38,40,41,42,43,159) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,HouseholdID,SurveyYear,PSUID,W2,OutCom_B02ID,HHIncome2002_B02ID,AddressType_B01ID,Ten1_B02ID,Landlord_B01ID,ResLength_B01ID,...,NumCarVan,WalkBus_B01ID,Getbus_B01ID,WalkRail_B01ID,WalkRailAlt_B01ID,HRPWorkStat_B02ID,HRPSEGWorkStat_B01ID,HHoldOAClass2011_B03ID,Settlement2011EW_B03ID,Settlement2011EW_B04ID
0,2002000001,2002,2002000001,0.987670,-10,2.0,3,1,-10,3,...,2.0,1,5,1,-9,1,3,-10,1,1.0
1,2002000002,2002,2002000001,1.013688,-10,1.0,3,1,-10,8,...,0.0,1,5,2,-9,3,5,-10,1,1.0
2,2002000003,2002,2002000001,1.254002,-10,3.0,3,1,-10,8,...,2.0,1,4,3,-9,1,3,-10,1,1.0
3,2002000004,2002,2002000001,1.098194,-10,3.0,3,1,-10,3,...,2.0,1,4,3,-9,1,3,-10,1,1.0
4,2002000005,2002,2002000001,0.911825,-10,3.0,2,1,-10,1,...,3.0,1,5,1,-9,1,1,-10,1,1.0


In [4]:
participation_mapping = dict(zip(hh_in.HouseholdID, hh_in.OutCom_B02ID))
weight_mapping = dict(zip(hh_in.HouseholdID, hh_in.W2))

## Load person data

In [5]:
persons_in = pd.read_csv(
    individuals_csv,
    sep='\t',
    usecols=['SurveyYear', 'IndividualID', 'HouseholdID', 'PSUID', 'VehicleID',
       'PersNo', 'Age_B01ID', 'OfPenAge_B01ID', 'Sex_B01ID', 'EdAttn1_B01ID',
       'EdAttn2_B01ID', 'EdAttn3_B01ID', 'DrivLic_B02ID', 'CarAccess_B01ID',
       'DrivDisable_B01ID', 'WkPlace_B01ID', 'ES2000_B01ID', 'NSSec_B03ID',
       'SC_B01ID', 'Stat_B01ID', 'SVise_B01ID', 'EcoStat_B02ID',
       'PossHom_B01ID']
)
persons_in.head()

,SurveyYear,IndividualID,HouseholdID,PSUID,VehicleID,PersNo,Age_B01ID,OfPenAge_B01ID,Sex_B01ID,EdAttn1_B01ID,...,CarAccess_B01ID,DrivDisable_B01ID,WkPlace_B01ID,ES2000_B01ID,NSSec_B03ID,SC_B01ID,Stat_B01ID,SVise_B01ID,EcoStat_B02ID,PossHom_B01ID
0,2002,2002000001,2002000001,2002000001,2002000001,1,13,2,2,-10,...,2,-9,1,7,2,3,1,2,2,2
1,2002,2002000002,2002000001,2002000001,2002000002,2,13,2,1,-10,...,2,-9,1,7,3,4,1,2,1,2
2,2002,2002000003,2002000001,2002000001,,3,4,2,1,-10,...,4,-9,-9,-9,-9,-9,-9,-9,-9,-9
3,2002,2002000004,2002000001,2002000001,,4,2,2,2,-10,...,4,-9,-9,-9,-9,-9,-9,-9,-9,-9
4,2002,2002000005,2002000002,2002000001,,1,18,1,2,-10,...,6,2,-9,6,1,3,1,1,4,-9


## load trip data
Pos. = 1	Variable = TripID	Variable label = Trip unique ID - Created in SQL
This variable is  numeric, the SPSS measurement level is SCALE
	Value label information for TripID

Pos. = 2	Variable = SurveyYear	Variable label = Survey year - actual year
This variable is  numeric, the SPSS measurement level is SCALE
	Value label information for SurveyYear

Pos. = 3	Variable = DayID	Variable label = ID given to all trips made by an individual on a given travel day - Created in SQL
This variable is  numeric, the SPSS measurement level is SCALE
	Value label information for DayID

Pos. = 4	Variable = IndividualID	Variable label = Individual unique ID - Created in SQL
This variable is  numeric, the SPSS measurement level is SCALE
	Value label information for IndividualID

Pos. = 5	Variable = HouseholdID	Variable label = Household unique ID - Created in SQL
This variable is  numeric, the SPSS measurement level is SCALE
	Value label information for HouseholdID

Pos. = 6	Variable = PSUID	Variable label = PSU unique ID - Created in SQL
This variable is  numeric, the SPSS measurement level is SCALE
	Value label information for PSUID

Pos. = 7	Variable = PersNo	Variable label = Person number within the household
This variable is  numeric, the SPSS measurement level is SCALE
	Value label information for PersNo

Pos. = 8	Variable = TravDay	Variable label = Day of the travel week (1-7)
This variable is  numeric, the SPSS measurement level is SCALE
	Value label information for TravDay

Pos. = 9	Variable = JourSeq	Variable label = Journey number on a given travel day
This variable is  numeric, the SPSS measurement level is SCALE
	Value label information for JourSeq


In [6]:
travel_diaries_in = pd.read_csv(
    trips_csv,
    sep='\t',
    usecols=['TripID', 'SurveyYear', 'DayID', 'IndividualID', 'HouseholdID', 'PSUID',
       'PersNo', 'TravDay', 'JourSeq', 'ShortWalkTrip_B01ID', 'NumStages',
       'MainMode_B04ID', 'TripPurpFrom_B01ID',
        'TripPurpTo_B01ID', 'TripPurpose_B04ID',
       'TripStart', 'TripEnd', 'TripOrigUA2009_B01ID', 'TripDestUA2009_B01ID'],
#     dtype={"W5": np.float64,}
)

# travel_diaries_in.W5 = pd.to_numeric(travel_diaries_in.W5, errors='coerce')

travel_diaries_in.head()

,TripID,SurveyYear,DayID,IndividualID,HouseholdID,PSUID,PersNo,TravDay,JourSeq,ShortWalkTrip_B01ID,NumStages,MainMode_B04ID,TripPurpose_B04ID,TripPurpFrom_B01ID,TripPurpTo_B01ID,TripStart,TripEnd,TripOrigUA2009_B01ID,TripDestUA2009_B01ID
0,2002000001,2002,2002000002,2002000001,2002000001,2002000001,1,2,1,2,1,4,7,23,10,675,683,-10,-10
1,2002000002,2002,2002000002,2002000001,2002000001,2002000001,1,2,2,2,1,4,6,10,8,720,735,-10,-10
2,2002000003,2002,2002000002,2002000001,2002000001,2002000001,1,2,3,2,1,4,7,8,10,770,780,-10,-10
3,2002000004,2002,2002000002,2002000001,2002000001,2002000001,1,2,4,2,1,12,7,10,23,1110,1130,-10,-10
4,2002000005,2002,2002000003,2002000001,2002000001,2002000001,1,3,1,2,1,4,7,23,10,760,770,-10,-10


In [7]:
travel_diaries_in['participation'] = travel_diaries_in.HouseholdID.map(participation_mapping)
travel_diaries_in['hh_weight'] = travel_diaries_in.HouseholdID.map(weight_mapping)

In [8]:
travel_diaries = travel_diaries_in.loc[travel_diaries_in.participation.isin([1,2])]

## PAM schema
    pid - person ID
    hid - household ID
    seq - trip sequence number
    hzone - household zone
    ozone - trip origin zone
    dzone - trip destination zone
    purp - trip purpose
    mode - trip mode
    tst - trip start time (minutes)
    tet - trip end time (minutes)
    freq - weighting for representative population

In [9]:
travel_diaries.rename(
    columns={  # rename data
        'JourSeq': 'seq',
        'TripOrigUA2009_B01ID': 'ozone',
        'TripDestUA2009_B01ID': 'dzone',
        'TripPurpFrom_B01ID': 'opurp',
        'TripPurpTo_B01ID': 'dpurp',
        'MainMode_B04ID': 'mode',
        'tst': 'TripStart',
        'tet': 'TripEnd',
    },
                inplace=True)

travel_diaries.head()

/Users/fred.shone/.pyenv/versions/jupyter3/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,TripID,SurveyYear,DayID,IndividualID,HouseholdID,PSUID,PersNo,TravDay,seq,ShortWalkTrip_B01ID,...,mode,TripPurpose_B04ID,opurp,dpurp,TripStart,TripEnd,ozone,dzone,participation,hh_weight
4006199,2016000001,2016,2016000001,2016000001,2016000001,2016000001,1,1,1,2,...,1,1,23,1,460,480,260,260,1,1.454712
4006200,2016000002,2016,2016000001,2016000001,2016000001,2016000001,1,1,2,2,...,1,1,1,23,1020,1040,260,260,1,1.454712
4006201,2016000003,2016,2016000002,2016000001,2016000001,2016000001,1,2,1,2,...,1,1,23,1,460,480,260,260,1,1.454712
4006202,2016000004,2016,2016000002,2016000001,2016000001,2016000001,1,2,2,2,...,1,1,1,23,1020,1040,260,260,1,1.454712
4006203,2016000005,2016,2016000003,2016000001,2016000001,2016000001,1,3,1,2,...,1,1,23,1,460,480,260,260,1,1.454712


In [10]:
travel_diaries.dtypes

TripID                   int64
SurveyYear               int64
DayID                    int64
IndividualID             int64
HouseholdID              int64
PSUID                    int64
PersNo                   int64
TravDay                  int64
seq                      int64
ShortWalkTrip_B01ID      int64
NumStages                int64
mode                     int64
TripPurpose_B04ID        int64
opurp                    int64
dpurp                    int64
TripStart               object
TripEnd                 object
ozone                    int64
dzone                    int64
participation            int64
hh_weight              float64
dtype: object

In [11]:
def check_uniques(df):
    for c in df.columns:
        print(c)
        n = df[c].nunique()
        if n < 1000:
            print(df[c].unique())

In [12]:
check_uniques(travel_diaries)

TripID
SurveyYear
[2016 2017]
DayID
IndividualID
HouseholdID
PSUID
PersNo
[ 1  2  3  4  5  6  7  8  9 10]
TravDay
[1 2 3 4 5 7 6]
seq
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]
ShortWalkTrip_B01ID
[2 1]
NumStages
[1 3 2 4 5 6 7 8]
mode
[ 1 11  4  8  9  6  3 12 13  2 10  7  5]
TripPurpose_B04ID
[1 4 7 6 2 8 5 3]
opurp
[23  1  4 10  8  2 14 11 15 22  9  5 13  3  6 12 20 21 17 18 16  7 19]
dpurp
[ 1 23  4 10  8  2 14 11 15 22  9  5 13  3  6 12 20 21 17 18 16  7 19]
TripStart
TripEnd
ozone
[260 151 540 153 331 520 480 400 131 530 370 800 112 500 231 840 230 360
 320 820 130 290 125 390 111 420 421 380 550 163 162 152 340 341 190 450
 342 470 350 154 210 460 560 101 164 161 510 220 221 180 113 270 490 440
 140 304 481 191 451 301 330 280 123 410 170 830 241 250 102 121 615 104
 810 271 561 124 430 431 303 240 212 714 242 103 126 122 211 272 620 302
 616 200 332 321 281 141 202 201 609 610 613 603 715 604 602 605 607 612
 622 601 716 720 701 702 608 310 606 618 611 705 

## Clean out incomplete plans

In [13]:
def remove_broken_plans(plan):
    if plan.isnull().values.any():
        return None
    for col in ['ozone', 'dzone']:
        if -8 in list(plan[col]):
            return None
    return plan

In [14]:
clean_travel_diaries = travel_diaries.groupby(
    ['IndividualID', 'TravDay']
).apply(
    remove_broken_plans
).reset_index(drop=True)

In [15]:
clean_travel_diaries.head()

,TripID,SurveyYear,DayID,IndividualID,HouseholdID,PSUID,PersNo,TravDay,seq,ShortWalkTrip_B01ID,...,mode,TripPurpose_B04ID,opurp,dpurp,TripStart,TripEnd,ozone,dzone,participation,hh_weight
0,2016000001,2016,2016000001,2016000001,2016000001,2016000001,1,1,1,2,...,1,1,23,1,460,480,260,260,1,1.454712
1,2016000002,2016,2016000001,2016000001,2016000001,2016000001,1,1,2,2,...,1,1,1,23,1020,1040,260,260,1,1.454712
2,2016000003,2016,2016000002,2016000001,2016000001,2016000001,1,2,1,2,...,1,1,23,1,460,480,260,260,1,1.454712
3,2016000004,2016,2016000002,2016000001,2016000001,2016000001,1,2,2,2,...,1,1,1,23,1020,1040,260,260,1,1.454712
4,2016000005,2016,2016000003,2016000001,2016000001,2016000001,1,3,1,2,...,1,1,23,1,460,480,260,260,1,1.454712


In [16]:
print(len(travel_diaries))
print(len(clean_travel_diaries))

442964
442917


## Reweight and Split Days

In [17]:
# reweight and split ids for unique days

def reweight(group):
    """
    Reweight based on multiple diary days, ie if an agent has two diary days, we will treat these as
    two unique agents, so we half the original weighting
    """
    group['freq'] = group.hh_weight / group.DayID.nunique()
    return group

trips = clean_travel_diaries.groupby('IndividualID').apply(reweight)
trips['pid'] = [f"{p}_{d}" for p, d in zip(trips.IndividualID, trips.TravDay)]
trips['hid'] = [f"{h}_{d}" for h, d in zip(trips.HouseholdID, trips.TravDay)]

In [18]:
trips.head()

,TripID,SurveyYear,DayID,IndividualID,HouseholdID,PSUID,PersNo,TravDay,seq,ShortWalkTrip_B01ID,...,dpurp,TripStart,TripEnd,ozone,dzone,participation,hh_weight,freq,pid,hid
0,2016000001,2016,2016000001,2016000001,2016000001,2016000001,1,1,1,2,...,1,460,480,260,260,1,1.454712,0.242452,2016000001_1,2016000001_1
1,2016000002,2016,2016000001,2016000001,2016000001,2016000001,1,1,2,2,...,23,1020,1040,260,260,1,1.454712,0.242452,2016000001_1,2016000001_1
2,2016000003,2016,2016000002,2016000001,2016000001,2016000001,1,2,1,2,...,1,460,480,260,260,1,1.454712,0.242452,2016000001_2,2016000001_2
3,2016000004,2016,2016000002,2016000001,2016000001,2016000001,1,2,2,2,...,23,1020,1040,260,260,1,1.454712,0.242452,2016000001_2,2016000001_2
4,2016000005,2016,2016000003,2016000001,2016000001,2016000001,1,3,1,2,...,1,460,480,260,260,1,1.454712,0.242452,2016000001_3,2016000001_3


In [21]:
def expand_days(
    trips,
    target,
    trips_on='Diary_number',
    target_on='Diary_number',
    new_id='pid',
    trim=True
):
    """
    Expand target df based on mapping between trips traget_on and new_id.
    Set index to new_id.
    """
    print("Building mapping.")
    mapping = {}
    for i, person in trips.groupby(target_on):
        mapping[i] = list(set(person[new_id]))
    n = len(mapping)
    
    if trim:
        print("Trimming target.")
        selection = set(trips[trips_on])
        target = target.loc[target[target_on].isin(selection)]
    
    expanded = pd.DataFrame()
    for p, (i, ids) in enumerate(mapping.items()):
        if not p % 10:
            print(f"Building expanded data {p}/{n}", end='\r', flush=True)
        for idx in ids:
            split = target.loc[target[target_on] == i]
            split[new_id] = idx
            expanded = expanded.append(split)
    expanded.set_index(new_id, inplace=True)
    print(f"Done")
    return expanded

In [22]:
hhs = expand_days(
    trips,
    hh_in,
    trips_on='HouseholdID',
    target_on='HouseholdID',
    new_id='hid'
)

Building mapping.
Trimming target.


/Users/fred.shone/.pyenv/versions/jupyter3/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [23]:
hhs.head()

,HouseholdID,SurveyYear,PSUID,W2,OutCom_B02ID,HHIncome2002_B02ID,AddressType_B01ID,Ten1_B02ID,Landlord_B01ID,ResLength_B01ID,...,NumCarVan,WalkBus_B01ID,Getbus_B01ID,WalkRail_B01ID,WalkRailAlt_B01ID,HRPWorkStat_B02ID,HRPSEGWorkStat_B01ID,HHoldOAClass2011_B03ID,Settlement2011EW_B03ID,Settlement2011EW_B04ID
hid,,,,,,,,,,,,,,,,,,,,,
2016000001_2,2016000001,2016,2016000001,1.454712,1,1.0,3,1,-10,7,...,1.0,-10,-10,-10,-10,1,2,3,1,NaN
2016000001_1,2016000001,2016,2016000001,1.454712,1,1.0,3,1,-10,7,...,1.0,-10,-10,-10,-10,1,2,3,1,NaN
2016000001_4,2016000001,2016,2016000001,1.454712,1,1.0,3,1,-10,7,...,1.0,-10,-10,-10,-10,1,2,3,1,NaN
2016000001_7,2016000001,2016,2016000001,1.454712,1,1.0,3,1,-10,7,...,1.0,-10,-10,-10,-10,1,2,3,1,NaN
2016000001_5,2016000001,2016,2016000001,1.454712,1,1.0,3,1,-10,7,...,1.0,-10,-10,-10,-10,1,2,3,1,NaN


In [ ]:
people = expand_days(
    trips,
    persons_in,
    trips_on='IndividualID',
    target_on='IndividualID',
    new_id='pid'
)

Building mapping.
Trimming target.


/Users/fred.shone/.pyenv/versions/jupyter3/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
people.head()

## Build Mappings and apply to common fields

In [ ]:
def string_to_dict(string):
    """used to build dicts from cut and paste rtf dictionaries"""
    mapping = {}
    for line in string.split("\n"):
        _, v, l = line.split("\t")
        v = v.split(" = ")[1]
        l = l.split(" = ")[1]
        mapping[float(v)] = str(l)
    return mapping

In [ ]:
mode_mapping = {
    1.0: 'Walk',
     2.0: 'Bicycle',
     3.0: 'Car/van driver',
     4.0: 'Car/van passenger',
     5.0: 'Motorcycle',
     6.0: 'Other private transport',
     7.0: 'Bus', #Bus in London',
     8.0: 'Bus', #'Other local bus',
     9.0: 'Coach', #'Non-local bus',
     10.0: 'London Underground',
     11.0: 'Surface Rail',
     12.0: 'Taxi/minicab',
     13.0: 'Other public transport',
     -10.0: 'DEAD',
     -8.0: 'NA'
}

In [ ]:
purp_mapping = {
    1.0: 'Work',
     2.0: 'In course of work',
     3.0: 'Education',
     4.0: 'Food shopping',
     5.0: 'Non food shopping',
     6.0: 'Personal business medical',
     7.0: 'Personal business eat/drink',
     8.0: 'Personal business other',
     9.0: 'Eat/drink with friends',
     10.0: 'Visit friends',
     11.0: 'Other social',
     12.0: 'Entertain/ public activity',
     13.0: 'Sport: participate',
     14.0: 'Holiday: base',
     15.0: 'Day trip/just walk',
     16.0: 'Other non-escort',
     17.0: 'Escort home',
     18.0: 'Escort work',
     19.0: 'Escort in course of work',
     20.0: 'Escort education',
     21.0: 'Escort shopping/personal business',
     22.0: 'Other escort',
     23.0: 'Home',
     -10.0: 'DEAD',
     -8.0: 'NA'
}

In [ ]:
area_mapping = {
    520.0: 'Warwickshire',
     530.0: 'West Midlands',
     540.0: 'West Sussex',
     191.0: 'Derby',
     550.0: 'West Yorkshire',
     560.0: 'Wiltshire',
     561.0: 'Swindon',
     601.0: 'Isle of Anglesey',
     602.0: 'Gwynedd',
     603.0: 'Conwy',
     604.0: 'Denbighshire',
     605.0: 'Flintshire',
     606.0: 'Wrexham',
     607.0: 'Powys',
     608.0: 'Ceredigion',
     609.0: 'Pembrokeshire',
     610.0: 'Carmarthenshire',
     611.0: 'Swansea',
     612.0: 'Neath and Port Talbot',
     101.0: 'Bath and N.E. Somerset',
     614.0: 'Vale of Glamorgan',
     103.0: 'North Somerset',
     616.0: 'Rhondda, Cynon, Taff',
     617.0: 'Merthyr Tydfil',
     618.0: 'Caerphilly',
     615.0: 'Cardiff',
     620.0: 'Torfaen',
     621.0: 'Monmouthshire',
     622.0: 'Newport',
     111.0: 'Luton',
     112.0: 'Bedford',
     104.0: 'South Gloucestershire',
     190.0: 'Derbyshire',
     121.0: 'Bracknell Forest',
     122.0: 'Newbury',
     123.0: 'Reading',
     124.0: 'Slough',
     125.0: 'Windsor & Maidenhead',
     126.0: 'Wokingham',
     130.0: 'Buckinghamshire',
     131.0: 'Milton Keynes',
     140.0: 'Cambridgeshire',
     141.0: 'Peterborough',
     151.0: 'Halton',
     152.0: 'Warrington',
     153.0: 'Cheshire East',
     154.0: 'Cheshire West and Chester',
     161.0: 'Hartlepool',
     162.0: 'Middlesbrough',
     163.0: 'Redcar & Cleveland',
     164.0: 'Stockton-on-Tees',
     113.0: 'Bedfordshire, Central',
     170.0: 'Cornwall & Isles of Scilly',
     200.0: 'Devon',
     180.0: 'Cumbria',
     201.0: 'Plymouth',
     701.0: 'Aberdeen City',
     702.0: 'Aberdeenshire',
     703.0: 'Angus',
     704.0: 'Argyll and Bute',
     705.0: 'Scottish Borders',
     706.0: 'Clackmannanshire',
     707.0: 'West Dunbartonshire',
     708.0: 'Dumfries and Galloway',
     709.0: 'Dundee, City of',
     710.0: 'East Ayrshire',
     711.0: 'East Dunbartonshire',
     712.0: 'East Lothian',
     713.0: 'East Renfrewshire',
     714.0: 'Edinburgh, City of',
     715.0: 'Falkirk',
     716.0: 'Fife',
     717.0: 'Glasgow, City of',
     718.0: 'Highland',
     719.0: 'Inverclyde',
     720.0: 'Midlothian',
     721.0: 'Moray',
     210.0: 'Dorset',
     211.0: 'Bournemouth',
     212.0: 'Poole',
     725.0: 'Perth and Kinross',
     726.0: 'Renfrewshire',
     727.0: 'Shetland Islands',
     728.0: 'South Ayrshire',
     729.0: 'South Lanarkshire',
     730.0: 'Stirling',
     731.0: 'West Lothian',
     732.0: 'Western Isles',
     221.0: 'Darlington',
     230.0: 'East Sussex',
     231.0: 'Brighton and Hove',
     722.0: 'North Ayrshire',
     240.0: 'Essex - area outside M25',
     241.0: 'Southend on Sea',
     242.0: 'Thurrock',
     723.0: 'North Lanarkshire',
     724.0: 'Orkney Islands',
     250.0: 'Gloucestershire',
     202.0: 'Torbay',
     260.0: 'Greater Manchester',
     270.0: 'Hampshire',
     271.0: 'Portsmouth',
     272.0: 'Southampton',
     280.0: 'Worcestershire',
     281.0: 'Herefordshire',
     800.0: 'Inner London - excluding Central London',
     290.0: 'Hertfordshire - area outside M25',
     220.0: 'Durham',
     810.0: 'Essex - area within M25',
     301.0: 'East Riding of Yorkshire',
     302.0: 'Kingston upon Hull, City of',
     303.0: 'North East Lincolnshire',
     304.0: 'North Lincolnshire',
     820.0: 'Hertfordshire - area within M25',
     310.0: 'Isle of Wight',
     830.0: 'Kent - area within M25',
     320.0: 'Kent - area outside M25',
     321.0: 'Medway Towns',
     840.0: 'Surrey - area within M25',
     330.0: 'Lancashire',
     331.0: 'Blackburn with Darwen',
     332.0: 'Blackpool',
     340.0: 'Leicestershire',
     341.0: 'Leicester',
     342.0: 'Rutland',
     350.0: 'Lincolnshire',
     360.0: 'London Central',
     370.0: 'Outer London',
     380.0: 'Merseyside',
     390.0: 'Norfolk',
     400.0: 'Northamptonshire',
     410.0: 'Northumberland',
     420.0: 'North Yorkshire',
     421.0: 'York',
     430.0: 'Nottinghamshire',
     431.0: 'Nottingham',
     440.0: 'Oxfordshire',
     450.0: 'Shropshire',
     451.0: 'The Wrekin',
     460.0: 'Somerset',
     470.0: 'South Yorkshire',
     480.0: 'Staffordshire',
     481.0: 'Stoke-on-Trent',
     490.0: 'Suffolk',
     613.0: 'Bridgend',
     619.0: 'Blaenau Gwent',
     500.0: 'Surrey - area outside M25',
     -10.0: 'DEAD',
     -9.0: 'DNA',
     -8.0: 'NA',
     102.0: 'Bristol, City of',
     510.0: 'Tyne and Wear'
}

In [ ]:
areas = gp.

In [ ]:
trips.columns

In [ ]:
trips['mode'] = trips['mode'].map(mode_mapping)
trips['oact'] = trips['opurp'].map(purp_mapping)
trips['dact'] = trips['dpurp'].map(purp_mapping)


In [ ]:
def remove_broken_plans(plan):
    if plan.isnull().values.any():
        return None
    for col in ['ozone', 'dzone', 'hzone']:
        if 'NI' in list(plan[col]):
            return None
        if 'WRONG FORMAT' in list(plan[col]):
            return None
    return plan

In [ ]:
hh_ids = set(travel_diaries_in.HouseholdID)
len(hh_ids)

In [ ]:
len(hh_ids - set(both_hh_ids))

In [ ]:
travel_diaries_in.loc[travel_diaries_in.HouseholdID == 2002000001]

In [ ]:
travel_diaries_in['participation'] = travel_diaries_in.HouseholdID.map(participation_mapping)

In [ ]:
travel_diaries = travel_diaries_in.loc[travel_diaries_in.participation.isin([1,2])]

In [ ]:
travel_diaries.head()

In [ ]:
len(travel_diaries)

In [ ]:
travel_diaries.loc[travel_diaries.participation == 2].head()

In [ ]:
set(travel_diaries.HouseholdID) - all_hh_ids

In [ ]:
travel_diaries_in.SurveyYear.unique()

In [ ]:
travel_diaries_in.head()

In [ ]:
tds_2017 = travel_diaries_in.loc[travel_diaries_in.SurveyYear == 2017, :]
tds_2017.head()

In [ ]:
tds_2017['uid'] = [str(x) + '_' + str(y) for x, y in zip(tds_2017['IndividualID'], tds_2017['DayID'])]

In [ ]:
tds_2017.head(20)

In [ ]:
households_in = pd.read_csv(households_csv, sep='\t')
households_in.head()

In [ ]:
individuals_in = pd.read_csv(individuals_csv, sep='\t')
individuals_in.head()

In [ ]:
travel_diaries_in

In [ ]:
travel_diaries_in.columns

In [ ]:
travel_diaries = tds_2017[
    [
        'uid',
        'HouseholdID',
        'JourSeq',
        'TripPurpFrom_B01ID',
        'TripOrigUA2009_B01ID',
        'TripDestUA2009_B01ID',
        'TripPurpose_B01ID',
        'MainMode_B04ID',
        'TripStart',
        'TripEnd',
        'W5'
    ]
]

In [ ]:
travel_diaries.head(20)

In [ ]:
travel_diaries.groupby('uid').

In [ ]:
travel_diaries.rename(columns={
    'HouseholdID': 'hid',
    'JourSeq': 'seq',
    'HHoldGOR_B02ID' : 'hzone',
    'TripPurpose_B02ID' : 'purp', 
    'MainMode_B03ID' : 'mode',
    'TripStart' : 'tst',
    'TripEnd' : 'tet'
})